In [4]:
import numpy as np

def detect_ssvep_fft_window(x, sample_rate=375, freqs=[10.5, 12, 15.2, 18.1], band=0.5):
    """
    x: EEG window, shape (C, T)
    """
    C, T = x.shape

    # FFT
    fft_vals = np.fft.rfft(x, axis=-1)
    power = np.abs(fft_vals)**2

    freq_axis = np.fft.rfftfreq(T, 1.0/sample_rate)

    scores = []
    for f in freqs:
        mask = (freq_axis >= f - band) & (freq_axis <= f + band)
        score = power[:, mask].mean()          # mean over channels + band
        scores.append(score)

    return np.argmax(scores), scores

def evaluate_fft_baseline(X, y, sample_rate=375):
    n_windows = X.shape[0]
    print(f"Evaluating FFT baseline on {n_windows} windows...")

    target_freqs = [10.5, 12, 15.2, 18.1]
    # IMPORTANT: remove "0" class for now — FFT cannot detect "no frequency"

    preds = []
    for i in range(n_windows):
        x = X[i]                    # (8, 375)
        pred_idx, _ = detect_ssvep_fft_window(
            x,
            sample_rate=sample_rate,
            freqs=target_freqs
        )
        preds.append(pred_idx)

    preds = np.array(preds)
    accuracy = (preds == y).mean()
    return accuracy, preds

params ={
    "data_path" : "datasets/numpy/ssvep_5_nofilter_GMCGA_no01_4labels_250hz_1.5_window_size.npz",
}
print(f"Loading data from: {params['data_path']}")
data = np.load(params["data_path"])
X_all, y_all = data["X"], data["y"]
print(f"Data loaded: X={X_all.shape}, y={y_all.shape}")

acc, preds = evaluate_fft_baseline(X_all, y_all)
print("FFT accuracy:", acc)


Loading data from: datasets/numpy/ssvep_5_nofilter_GMCGA_no01_4labels_250hz_1.5_window_size.npz
Data loaded: X=(1576, 8, 375), y=(1576,)
Evaluating FFT baseline on 1576 windows...
FFT accuracy: 0.2233502538071066
